# Análise de Conteúdo Visual com LLMs

Este notebook implementa o pipeline de análise de imagens utilizando modelos multimodais (GPT, Gemini, Claude). Ele carrega dados de anotação, submete imagens aos modelos, processa as respostas e gera visualizações comparativas das bounding boxes.

## 1. Importações e Configuração

In [ ]:
import os
import json
import math
import requests
import openai
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv

# Adicionar o diretório atual ao path para garantir importação dos módulos locais
import sys
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

# Importar clientes LLM do arquivo llm_clients.py
try:
    from llm_clients import analyze_with_gpt, analyze_with_gemini, analyze_with_claude
except ImportError:
    print("Erro: O arquivo 'llm_clients.py' não foi encontrado no diretório atual.")

# Carregar variáveis de ambiente
# Ajuste o caminho conforme necessário para o seu ambiente
load_dotenv(dotenv_path="e:/2025/ProjectEyeLLM/.env")

# Configuração da API da OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
print(f"Chave OpenAI carregada: {'Sim' if openai.api_key else 'Não'}")
print(f"Chave Gemini carregada: {'Sim' if os.getenv('GEMINI_API_KEY') else 'Não'}")
print(f"Chave Claude carregada: {'Sim' if os.getenv('CLAUDE_API_KEY') else 'Não'}")

## 2. Funções de Manipulação de Arquivos

In [ ]:
def ler_arquivo_json(caminho_arquivo):
    """
    Lê um arquivo JSON e retorna os dados como um objeto JSON.
    """
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"Erro ao ler o arquivo JSON: {e}")
        return []

def listar_e_ler_json(pasta_dados, filtro_subject=None, filtro_nome_imagem=None):
    """
    Lista todos os arquivos JSON na raiz da pasta especificada, lê os registros e aplica os filtros.
    """
    registros_filtrados = []

    try:
        # Listar todos os arquivos JSON na pasta
        arquivos_json = [f for f in os.listdir(pasta_dados) if f.endswith('.json')]

        # Processar cada arquivo JSON
        for arquivo in arquivos_json:
            caminho_arquivo = os.path.join(pasta_dados, arquivo)
            registros = ler_arquivo_json(caminho_arquivo)

            # Verificar se o arquivo JSON contém uma lista de registros
            if isinstance(registros, list):
                for registro in registros:
                    try:
                        # Aplicar filtros
                        if filtro_subject is not None and registro.get("subject") != filtro_subject:
                            continue
                        if filtro_nome_imagem is not None and registro.get("name") != filtro_nome_imagem:
                            continue

                        # Adicionar registro filtrado à lista
                        registros_filtrados.append(registro)
                    except Exception as e:
                        print(f"Erro ao processar registro no arquivo {arquivo}: {e}")
            else:
                print(f"O arquivo {arquivo} não contém uma lista de registros.")
    except Exception as e:
        print(f"Erro ao listar ou ler arquivos JSON na pasta {pasta_dados}: {e}")

    return registros_filtrados

def ler_prompt(caminho_prompt):
    """Lê o conteúdo de um arquivo de texto (prompt)."""
    try:
        with open(caminho_prompt, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except FileNotFoundError:
        print(f"Arquivo de prompt não encontrado: {caminho_prompt}")
        return None

## 3. Funções de Processamento e Métricas

In [ ]:
def gerar_texto(template_path, dados):
    """Preenche o template do prompt com os dados fornecidos."""
    try:
        with open(template_path, "r", encoding="utf-8") as f:
            texto = f.read()

        for chave, valor in dados.items():
            marcador = f"{{{{{chave}}}}}"
            texto = texto.replace(marcador, str(valor))
        
        return texto
    except Exception as e:
        print(f"Erro ao gerar texto do prompt: {e}")
        return ""

def calcular_distancia_bbox(bbox_dataset, bbox_modelo):
    """
    Calcula a distância euclidiana entre os centros de duas bounding boxes.
    Args:
        bbox_dataset (list): [x, y, w, h] da anotação do dataset.
        bbox_modelo (list): [x, y, w, h] da resposta do modelo.
    Returns:
        float: Distância em pixels entre os centros das caixas.
    """
    if not bbox_dataset or not bbox_modelo:
        return float('inf')

    # Centro da bbox do dataset
    cx_d = bbox_dataset[0] + bbox_dataset[2] / 2
    cy_d = bbox_dataset[1] + bbox_dataset[3] / 2

    # Centro da bbox do modelo
    cx_m = bbox_modelo[0] + bbox_modelo[2] / 2
    cy_m = bbox_modelo[1] + bbox_modelo[3] / 2

    # Distância Euclidiana
    return math.sqrt((cx_d - cx_m)**2 + (cy_d - cy_m)**2)

## 4. Funções de Visualização

In [ ]:
def salvar_resultado_visual(url_imagem, objetos_resposta, pasta_saida, nome_arquivo):
    """
    Desenha as caixas delimitadoras na imagem e salva o resultado.
    Nota: Baixa a imagem da URL para desenhar.
    """
    try:
        # Baixar a imagem da URL
        response = requests.get(url_imagem)
        img = Image.open(BytesIO(response.content))
        
        plt.figure(figsize=(10, 8))
        plt.imshow(img)
        plt.axis('off')

        # Desenhar as caixas delimitadoras
        for objeto in objetos_resposta:
            if "bbox" in objeto:
                x, y, largura, altura = objeto["bbox"]
                label = objeto.get("task", "object")
                plt.gca().add_patch(Rectangle((x, y), largura, altura, edgecolor='red', facecolor='none', linewidth=2))
                plt.text(x, y - 10, label, color='red', fontsize=10, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

        # Salvar a imagem com as caixas
        caminho_saida = os.path.join(pasta_saida, nome_arquivo)
        plt.savefig(caminho_saida)
        plt.close()
        print(f"Imagem com caixas salva em: {caminho_saida}")

    except Exception as e:
        print(f"Erro ao desenhar/salvar imagem: {e}")

## 5. Configurações e Variáveis

In [ ]:
# --- CONFIGURAÇÃO DO MODELO ---
NOME_MODELO = "GPT5_2" # Opções: "GPT5_2", "Gemini", "Claude"
# ------------------------------

# Identificar a raiz do projeto (assumindo estrutura: ProjectEyeLLM/EyeLLM/Modulo1/notebook.ipynb)
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, "..", ".."))
print(f"Raiz do projeto identificada: {project_root}")

# Caminhos absolutos baseados na raiz
pasta_dados = os.path.join(project_root, "dados")
pasta_saida_modelo = os.path.join(project_root, "resultados", NOME_MODELO)
caminho_prompt = os.path.join(project_root, "resultados", "Modulo1", "Prompts", "GPT5_2.txt")

# Filtros
filtro_subject = 2
filtro_nome_imagem = "000000001455.jpg"

## 6. Execução Principal

In [ ]:
# Criar a pasta de saída do modelo, se não existir
os.makedirs(pasta_saida_modelo, exist_ok=True)

# Ler os arquivos JSON
registros = listar_e_ler_json(pasta_dados, filtro_subject, filtro_nome_imagem)
print(f"Número de registros filtrados: {len(registros)}")

# Loop de Processamento
for registro in registros:
    try:
        # Construir a URL da imagem
        base_url = "https://raw.githubusercontent.com/TardellyCavalcante/ProjectEyeLLM/ad853fc7c12affef287f958f6490f15a4fb1a605/dados/imagens"
        url_imagem = f"{base_url}/{registro['task']}/{registro['name']}"
        print(f"\nProcessando: {registro['name']} (Task: {registro['task']})")

        # Validar a URL
        try:
            response = requests.head(url_imagem)
            if response.status_code != 200:
                print(f"Erro: URL da imagem não é acessível. Código: {response.status_code}")
                continue
        except Exception as e:
            print(f"Erro ao validar a URL da imagem: {e}")
            continue

        # Gerar o prompt
        dados_prompt = {"subject": registro['subject'], "name": registro['name']}
        if os.path.exists(caminho_prompt):
            prompt = gerar_texto(caminho_prompt, dados_prompt)
        else:
            print("Arquivo de prompt não encontrado. Usando prompt genérico.")
            prompt = f"Analise a imagem {registro['name']} e identifique os objetos."

        # --- CHAMADA DA API ---
        resposta = None
        if NOME_MODELO == "GPT5_2":
            resposta = analyze_with_gpt(os.getenv("OPENAI_API_KEY"), url_imagem, prompt)
        elif NOME_MODELO == "Gemini":
            resposta = analyze_with_gemini(os.getenv("GEMINI_API_KEY"), url_imagem, prompt)
        elif NOME_MODELO == "Claude":
            resposta = analyze_with_claude(os.getenv("CLAUDE_API_KEY"), url_imagem, prompt)
        
        if not resposta:
            print("Sem resposta da API.")
            continue

        # Parse da resposta (JSONL)
        linhas = [l for l in resposta.splitlines() if l.strip()]
        objetos = []
        for linha in linhas:
            try:
                objetos.append(json.loads(linha))
            except json.JSONDecodeError:
                print(f"Linha inválida ignorada: {linha}")

        # Salvar a resposta textual
        nome_base = f"{registro['subject']}_{os.path.splitext(registro['name'])[0]}"
        caminho_txt = os.path.join(pasta_saida_modelo, f"{nome_base}.txt")
        with open(caminho_txt, 'w', encoding='utf-8') as f:
            f.write(resposta)
        print(f"Texto salvo em: {caminho_txt}")

        # Calcular distâncias e métricas
        for objeto in objetos:
            if "bbox" in objeto and "bbox" in registro and objeto.get("task") == registro.get("task"):
                distancia = calcular_distancia_bbox(registro["bbox"], objeto["bbox"])
                print(f"Distância entre centros para '{registro.get('task')}': {distancia:.2f} pixels")

        # Desenhar e salvar resultado visual
        salvar_resultado_visual(url_imagem, objetos, pasta_saida_modelo, f"{nome_base}.png")

    except Exception as e:
        print(f"Erro ao processar o registro: {e}")